In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from time import sleep
import random
import datetime
from tqdm import tqdm
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# label - 구하고자 하는 카테고리별 고정 값 지정
labels = {
    "경제/정책": "economic-policy",
    "금융": "finance",
    "증시": "stock-market",
    "부동산": "real-estate",
    "소비자": "consumer",
    "국제경제": "international-economy"
}

In [3]:
# date
dt = datetime.datetime.now()  # 현재 날짜
pages = int(input("몇 페이지? > "))  # 몇 페이지를 부를지
print(dt)
date = dt.strftime("%Y%m%d")  # str로 변환
print(date)

pages_list = list()
for page in range(1, pages+1):  # 페이지 추가
    pages_list.append(page)
len(pages_list)

2023-08-20 15:29:37.427236
20230820


3

In [25]:
# title과 content 반환
def make_head_content(url_list, day, label):
    sub_df = pd.DataFrame()

    for url in url_list:
        r2 = requests.get(url) # 요청 받기
        soup2 = BeautifulSoup(r2.text, "html.parser")  # html 정보를 txt로 받아오기

        # 불필요 기호 제거
        pattern1 = '<[^>]*>|\t|\n|<b>|</b>|&apos;' # 불필요한 기호 제거

        # title
        title = soup2.select_one("header.title-article01 > h1.tit") # 기사제목 불러오기
        if title == None:
            title = None
        else:
            title = title.get_text() # 타이틀 텍스트로 받기

        article = soup2.select("div.scroller01 > article.story-news.article > p") # 기사본문 받기
        article = "".join([text.get_text() for text in article]) # 기사 본문들 text로 연결

        if article == None: 
            article = None
        else:
            article = re.sub(pattern=pattern1, repl='', string=str(article)) # 기사에서 특수부호 제거
            
        if len(article) < 300: # 기사 본문이 300자 이하인 기사는 제거
            title = None
            content = None
        for tag in ["[표]", "[인사]", "[코스피]", "[코스닥]", "[연합뉴스 이 시각 헤드라인]"]: # 내용이 아닌 알림과 관련된 기사 제거
            if tag in article:
                title=None
                content = None
        else:        
            # 전체 지문의 20%만 사용
            limit = int(round(len(article)*0.2, 0))
            content = article[:limit]+" ...중략..."

        sub = pd.DataFrame({"title": [title], "content": [content], "date": [day], "label": [label]}) # 데이터 추가
        sub_df = pd.concat([sub_df, sub], axis=0)

        sleep(random.uniform(1, 1.5))  # 막히지 않으려 약간의 변동 주기

    return sub_df

In [26]:
# 짧은 기간 사용 - 하루에 한 페이지
news_df = pd.DataFrame()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # 화면창 백그라운드
chrome_options.add_argument("--disable-gpu")  # 불필요한 gpu 사용 X
chrome_options.add_argument("--no-sandbox") # 보안기능 강화 해제
chrome_options.add_argument("--disable-setuid-sandbox") # 보안기능 비활성화
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 사용량 감소
#cnt = 1

def main(pages_list, news_df):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options) # 크롬 드라이버 사용

    for page in tqdm(pages_list):
        for label in labels:
            driver.get(f"https://www.yna.co.kr/economy/{labels[label]}/{page}") # url을 통한 접근
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.paging.paging-type01"))) ## 해당 요소가 나타날 때까지 대기

            # html 정보를 txt로 받아오기
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 각 기사별 링크 가져오기
            cnt = soup.select("div.list-type038 > ul.list > li > div.item-box01 > div.news-con > a")

            # url_list 생성
            url_list = [f"https:{link.attrs['href']}" for link in cnt]
            
            # 제목과 내용 반환
            sub_df = make_head_content(url_list, date, label)
            news_df = pd.concat([news_df, sub_df], axis=0)

    driver.quit()
    return news_df


news_df = main(pages_list, news_df)

100%|██████████| 3/3 [10:45<00:00, 215.17s/it]


In [28]:
news_df = news_df.dropna(axis=0) # null값 제거
news_df.reset_index(drop=True, inplace=True) # 인덱스 초기화

# id 생성
news_df["news_id"] = [f"news_{i}" for i in range(1, news_df.shape[0]+1)]

(398, 4)

In [33]:
# csv로 저장
news_df.to_csv(os.path.join(os.getcwd(), "data", "news_db.csv"), encoding="utf-8", index=False)